In [1]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit


2022-06-14 15:32:26.258323: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-14 15:32:26.258348: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-06-14 15:32:28.432816: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-14 15:32:28.433305: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-14 15:32:28.433380: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.s

In [2]:
from openfermion.chem import MolecularData
from openfermionpyscf import run_pyscf
import mindquantum as mq

In [3]:
# data = [["H", [0.0, 0.0, -0.6614]], ["H", [0.0, 0.0, 0.6614]]]
data = [["Li", [0, 0, 0]], ["H", [1, 0, 0]]]

In [4]:
molecule_of = MolecularData(
    geometry=data, 
    basis="sto3g", 
    multiplicity=1
)

molecule_of = run_pyscf(
    molecule_of, 
    run_scf=1, 
    run_ccsd=1, 
    run_fci=1
)

# print("Hartree-Fock energy: %20.16f Ha" % (molecule_of.hf_energy))
# print("CCSD energy: %20.16f Ha" % (molecule_of.ccsd_energy))
# print("FCI  energy: %20.16f Ha" % (molecule_of.fci_energy))

molecule_of.save()
molecule_file = molecule_of.filename
# print(molecule_file)

hartreefock_wfn_circuit = mq.Circuit([
    mq.X.on(i) for i in range(molecule_of.n_electrons)
])
print(hartreefock_wfn_circuit)

ansatz_circuit, \
init_amplitudes, \
ansatz_parameter_names, \
hamiltonian_QubitOp, \
n_qubits, \
n_electrons = mq.algorithm.generate_uccsd(molecule_file, th=-1)

total_circuit = hartreefock_wfn_circuit + ansatz_circuit
total_circuit.summary()
print("Number of parameters: %d" % (len(ansatz_parameter_names)))

q0: ──X──

q1: ──X──

q2: ──X──

q3: ──X──
ccsd:-7.784454825910822.
fci:-7.78446028003123.
============================Circuit Summary============================
|Total number of gates  : 15172.                                      |
|Parameter gates        : 640.                                        |
|with 44 parameters are : p0, p8, p1, p9, p2, p10, p3, p11, p4, p12...|
|Number qubit of circuit: 12                                          |
Number of parameters: 44


In [5]:
print(str(hamiltonian_QubitOp))

-3.9344 [] +
-0.0079 [X0 X1 Y2 Y3] +
-0.0034 [X0 X1 Y2 Z3 Z4 Y5] +
-0.0027 [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
-0.0034 [X0 X1 X3 X4] +
-0.0027 [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
-0.0049 [X0 X1 Y4 Y5] +
-0.0023 [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
-0.0023 [X0 X1 X5 Z6 Z7 Z8 Z9 X10] +
-0.0025 [X0 X1 Y6 Y7] +
-0.0025 [X0 X1 Y8 Y9] +
-0.0018 [X0 X1 Y10 Y11] +
0.0079 [X0 Y1 Y2 X3] +
0.0034 [X0 Y1 Y2 Z3 Z4 X5] +
0.0027 [X0 Y1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
-0.0034 [X0 Y1 Y3 X4] +
-0.0027 [X0 Y1 Y3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
0.0049 [X0 Y1 Y4 X5] +
0.0023 [X0 Y1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
-0.0023 [X0 Y1 Y5 Z6 Z7 Z8 Z9 X10] +
0.0025 [X0 Y1 Y6 X7] +
0.0025 [X0 Y1 Y8 X9] +
0.0018 [X0 Y1 Y10 X11] +
0.0238 [X0 Z1 X2] +
-0.0016 [X0 Z1 X2 X3 Z4 X5] +
0.0031 [X0 Z1 X2 X3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
-0.0016 [X0 Z1 X2 Y3 Z4 Y5] +
0.0031 [X0 Z1 X2 Y3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
-0.0037 [X0 Z1 X2 Z3] +
-0.0017 [X0 Z1 X2 X4 Z5 Z6 Z7 Z8 Z9 X10] +
0.0005 [X0 Z1 X2 Y4 Z5 Z6 Z7 Z8 Z9 Y10] +
0.0042 [

In [5]:
def parse_braces(w: str, lb: str, rb: str) -> str:
    i = w.find(lb)
    j = w.find(rb)
    return w[i+1:j]

def trans_hamiltonian(mq_hamiltonion, qreg):
    gate_map = {
        "X": cirq.ops.X,
        "Y": cirq.ops.Y,
        "Z": cirq.ops.Z,
    }
    ham = cirq.PauliSum()
    for l in str(mq_hamiltonion).splitlines():
        op = parse_braces(l, "[", "]")
        idx = l.find("[")
        x = float(l[:idx])

        if op == "":
            ham += x
            continue
        

        v = []
        for w in op.split(" "):
            g = gate_map[w[0]]
            idx = int(w[1:])
            v.append(g.on(qreg[idx]))
        
        ham += x * cirq.PauliString(*tuple(v))
    return ham

qreg = cirq.LineQubit.range(n_qubits)        

ham = trans_hamiltonian(hamiltonian_QubitOp, qreg)
# print(ham)

In [6]:
from mindquantum.core import gates as mgates
from mindquantum.core import Circuit as mcircuit

def trans_circuit_mindquantum_cirq(mcircuit: mcircuit, n_qubits: int, qreg, pr_table):
    import cirq

    circ = cirq.Circuit()

    def self_herm_non_params(gate):
        ctrls = gate.ctrl_qubits
        objs = gate.obj_qubits
        if ctrls:
            # must be CNOT
            circ.append([cirq.ops.CNOT.on(qreg[ctrls[0]], qreg[objs[0]])])
        else:
            # must be H
            gate_map = {
                "X": cirq.ops.X,
                "Y": cirq.ops.Y,
                "Z": cirq.ops.Z,
                "H": cirq.ops.H,
            }
            g = gate_map[gate.name.upper()]
            circ.append([g.on(qreg[objs[0]])])

    def params_gate_trans(gate, pr_table):
        gate_map = {
            "RX": cirq.ops.rx,
            "RY": cirq.ops.ry,
            "RZ": cirq.ops.rz,
        }
        objs = gate.obj_qubits
        if gate.ctrl_qubits:
            raise ValueError(f"Can't convert {gate} with params.")

        g = gate_map[gate.name.upper()]
        if gate.parameterized:
            # parameter
            # acc = None
            for k,v in gate.coeff.items():
                if k not in pr_table:
                    pr_table[k] = sympy.Symbol(k)
                circ.append([g(v*pr_table[k]).on(qreg[objs[0]])])
            #     if acc is None:
            #         acc = v * pr_table[k]
            #     else:
            #         acc += v * pr_table[k]
            # g = g(acc).on(qreg[objs[0]])
        else:
            # no parameter
            g = g(gate.coeff.const).on(qreg[objs[0]])
            circ.append([g])

    cnt1, cnt2 = 0, 0
    mcircuit = mcircuit.remove_barrier()
    # pr_table = dict()
    for g in mcircuit:
        if isinstance(g, (
            mgates.XGate, mgates.HGate
        )):
            cnt1 += 1
            self_herm_non_params(g)
        elif isinstance(g, (
            mgates.RX, mgates.RY, mgates.RZ
        )):
            cnt2 += 1
            params_gate_trans(g, pr_table)
        else:
            raise ValueError(f"Haven't implemented convertion for gate {g}")
    print(f"cnt1={cnt1}, cnt2={cnt2}")
    return circ

pr_table = dict()
circ = trans_circuit_mindquantum_cirq(total_circuit, n_qubits, qreg, pr_table)
# SVGCircuit(circ)

cnt1=9476, cnt2=3136


In [7]:
expectation_calculation = tfq.layers.Expectation(
    differentiator=tfq.differentiators.ForwardDifference(grid_spacing=0.01))


In [9]:
# expectation_calculation(circ,
#                         operators=ham,
#                         symbol_names=list(pr_table.keys()),
#                         symbol_values=[[0 for _ in range(len(pr_table))]])


In [8]:
theta = np.zeros((1, len(pr_table))).astype(np.float32)
theta_tensor = tf.convert_to_tensor(theta)
print(theta_tensor.shape)

iter_num = 20

for i in range(iter_num):
    with tf.GradientTape() as g:
        g.watch(theta_tensor)
        output = expectation_calculation(
            circ,
            operators=ham,
            symbol_names = list(pr_table.keys()),
            symbol_values=theta_tensor,
        )
        grad = g.gradient(output, theta_tensor)
        theta_tensor -= grad
        if i % 1 == 0:
            print(f"Step {i}: loss = {output[0]}")

(1, 44)
Step 0: loss = [-7.7656684]
Step 1: loss = [-4.487466]
Step 2: loss = [-2.4954274]
Step 3: loss = [-3.3900628]
Step 4: loss = [-3.38505]
Step 5: loss = [-3.844197]
Step 6: loss = [-4.5066338]
Step 7: loss = [-2.7017756]
Step 8: loss = [-3.6002488]
Step 9: loss = [-4.420015]
Step 10: loss = [-4.217026]
Step 11: loss = [-4.5410028]
